In [5]:
import numpy as np
import geog  ## package for geodesic functions
import shapely.geometry
import json
import geopandas as gpd
import overpass # Python package to interface with OSM Overpass API
import folium


####inputs
radius = 3 # radius in KM
starting_lat = 40.679044
starting_lon = -73.971655

crs = {'init': 'epsg:4326'}
#### create geometry based on a single point
p = shapely.geometry.Point([ starting_lon, starting_lat ])


#### generate polygon based on radius around point
n_points = 20
d = radius * 1000  # meters
angles = np.linspace(0, 360, n_points)
polygon = geog.propagate(p, angles, d)
#geoj = json.loads(json.dumps(shapely.geometry.mapping(shapely.geometry.Polygon(polygon))))  ### code to convert polygon to geoJSON 

# Create GeoDataFrame based on polygon around point
geoj= gpd.GeoDataFrame(crs=crs)
geoj['geometry'] = None
geoj.loc[0, 'geometry'] = shapely.geometry.Polygon(polygon)
outline_box_osm = ''

for i in [1,0,3,2]:
    outline_box_osm  = outline_box_osm + str(geoj['geometry'][0].bounds[i])+','

outline_box_osm=outline_box_osm[0:-1]
print (outline_box_osm)


with open('data.json', 'w') as outfile:
    json.dump(shapely.geometry.mapping(shapely.geometry.Polygon(polygon)),outfile)


api = overpass.API()
response = api.Get('node[\"amenity\"=\"restaurant\"][\"cuisine\"=\"pizza\"]('+outline_box_osm+')')

##print (response)


poly = gpd.read_file('data.json')
gs = gpd.GeoDataFrame.from_features(response['features'],crs=crs)

print (len(gs))

#print("Head of points data ",gs.head())

pointInPolys = gpd.sjoin(gs,geoj ,op='within')


print (len(pointInPolys))






import matplotlib.pyplot as plt
import mplleaflet

geoj.plot()
pointInPolys.plot()


zoom_level = (radius * -1) + 16




mapa  =  folium.Map([starting_lat, starting_lon ],  zoom_start=zoom_level,tiles='cartodbpositron')


#folium.GeoJson(geoj).add_to(m)
#folium.GeoJson(gs).add_to(m)

style_function = lambda x: {'fillColor': '#0000ff'}


##gj = folium.GeoJson(pointInPolys).add_to(m)

#gj.add_child(folium.Icon(color='green'))
#gj.add_child(folium.Popup('outline Popup on GeoJSON'))
#gj.add_to(m)
#m.save( 'geopandas_1.html')

#print (pointInPolys.head())

from folium import IFrame, Map
from folium.plugins import MarkerCluster


table = """
<!DOCTYPE html>
<html>
<head>
<style>
table {{
    width:100%;
}}
table, th, td {{
    border: 1px solid black;
    border-collapse: collapse;
}}
th, td {{
    padding: 5px;
    text-align: left;
}}
table#t01 tr:nth-child(odd) {{
    background-color: #eee;
}}
table#t01 tr:nth-child(even) {{
   background-color:#fff;
}}
</style>
</head>
<body>

<table id="t01">
  <tr>
    <td>Cuisine Type</td>
    <td>{}</td>
  </tr>
  <tr>
    <td>Restaurant Name</td>
    <td>{}</td>
  </tr>
</table>
</body>
</html>
""".format

import matplotlib
matplotlib.pyplot.close('all')
width, height = 310,110
popups, locations = [], []
for idx, row in pointInPolys.iterrows():
    locations.append([row['geometry'].y, row['geometry'].x])
    name = row['name'].encode('ascii', 'xmlcharrefreplace')

    iframe = IFrame(table('Pizza', name), width=width, height=height)
    popups.append(iframe)

t = folium.FeatureGroup(name='Thermoelectric')
t.add_child(MarkerCluster(locations=locations, popups=popups))
mapa.add_child(t)


folium.GeoJson(geoj).add_to(mapa)
mapa.save('geo2.html')

mapa

40.652156463789915,-74.00674783660311,40.705931461747404,-73.9360792949964


23
18


<div style="width:100%;"><div style="position:relative;width:100%;height:0;padding-bottom:60%;"><iframe src="data:text/html;charset=utf-8;base64,PCFET0NUWVBFIGh0bWw+CjxoZWFkPiAgICAKICAgIDxtZXRhIGh0dHAtZXF1aXY9ImNvbnRlbnQtdHlwZSIgY29udGVudD0idGV4dC9odG1sOyBjaGFyc2V0PVVURi04IiAvPgogICAgPHNjcmlwdD5MX1BSRUZFUl9DQU5WQVMgPSBmYWxzZTsgTF9OT19UT1VDSCA9IGZhbHNlOyBMX0RJU0FCTEVfM0QgPSBmYWxzZTs8L3NjcmlwdD4KICAgIDxzY3JpcHQgc3JjPSJodHRwczovL2Nkbi5qc2RlbGl2ci5uZXQvbnBtL2xlYWZsZXRAMS4yLjAvZGlzdC9sZWFmbGV0LmpzIj48L3NjcmlwdD4KICAgIDxzY3JpcHQgc3JjPSJodHRwczovL2FqYXguZ29vZ2xlYXBpcy5jb20vYWpheC9saWJzL2pxdWVyeS8xLjExLjEvanF1ZXJ5Lm1pbi5qcyI+PC9zY3JpcHQ+CiAgICA8c2NyaXB0IHNyYz0iaHR0cHM6Ly9tYXhjZG4uYm9vdHN0cmFwY2RuLmNvbS9ib290c3RyYXAvMy4yLjAvanMvYm9vdHN0cmFwLm1pbi5qcyI+PC9zY3JpcHQ+CiAgICA8c2NyaXB0IHNyYz0iaHR0cHM6Ly9jZG5qcy5jbG91ZGZsYXJlLmNvbS9hamF4L2xpYnMvTGVhZmxldC5hd2Vzb21lLW1hcmtlcnMvMi4wLjIvbGVhZmxldC5hd2Vzb21lLW1hcmtlcnMuanMiPjwvc2NyaXB0PgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL2Nkbi5qc2RlbGl2ci5uZXQvbnBtL2xlYWZsZXRAMS4yLjAvZGlzdC9sZWFmbGV0LmNzcyIvPgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL21heGNkbi5ib290c3RyYXBjZG4uY29tL2Jvb3RzdHJhcC8zLjIuMC9jc3MvYm9vdHN0cmFwLm1pbi5jc3MiLz4KICAgIDxsaW5rIHJlbD0ic3R5bGVzaGVldCIgaHJlZj0iaHR0cHM6Ly9tYXhjZG4uYm9vdHN0cmFwY2RuLmNvbS9ib290c3RyYXAvMy4yLjAvY3NzL2Jvb3RzdHJhcC10aGVtZS5taW4uY3NzIi8+CiAgICA8bGluayByZWw9InN0eWxlc2hlZXQiIGhyZWY9Imh0dHBzOi8vbWF4Y2RuLmJvb3RzdHJhcGNkbi5jb20vZm9udC1hd2Vzb21lLzQuNi4zL2Nzcy9mb250LWF3ZXNvbWUubWluLmNzcyIvPgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL2NkbmpzLmNsb3VkZmxhcmUuY29tL2FqYXgvbGlicy9MZWFmbGV0LmF3ZXNvbWUtbWFya2Vycy8yLjAuMi9sZWFmbGV0LmF3ZXNvbWUtbWFya2Vycy5jc3MiLz4KICAgIDxsaW5rIHJlbD0ic3R5bGVzaGVldCIgaHJlZj0iaHR0cHM6Ly9yYXdnaXQuY29tL3B5dGhvbi12aXN1YWxpemF0aW9uL2ZvbGl1bS9tYXN0ZXIvZm9saXVtL3RlbXBsYXRlcy9sZWFmbGV0LmF3ZXNvbWUucm90YXRlLmNzcyIvPgogICAgPHN0eWxlPmh0bWwsIGJvZHkge3dpZHRoOiAxMDAlO2hlaWdodDogMTAwJTttYXJnaW46IDA7cGFkZGluZzogMDt9PC9zdHlsZT4KICAgIDxzdHlsZT4jbWFwIHtwb3NpdGlvbjphYnNvbHV0ZTt0b3A6MDtib3R0b206MDtyaWdodDowO2xlZnQ6MDt9PC9zdHlsZT4KICAgIAogICAgICAgICAgICA8c3R5bGU+ICNtYXBfMTAzYzg4ZTI0YWRlNDEwOTg3ZTU4YjZmODU4NGQ1NWYgewogICAgICAgICAgICAgICAgcG9zaXRpb24gOiByZWxhdGl2ZTsKICAgICAgICAgICAgICAgIHdpZHRoIDogMTAwLjAlOwogICAgICAgICAgICAgICAgaGVpZ2h0OiAxMDAuMCU7CiAgICAgICAgICAgICAgICBsZWZ0OiAwLjAlOwogICAgICAgICAgICAgICAgdG9wOiAwLjAlOwogICAgICAgICAgICAgICAgfQogICAgICAgICAgICA8L3N0eWxlPgogICAgICAgIAogICAgPHNjcmlwdCBzcmM9Imh0dHBzOi8vY2RuanMuY2xvdWRmbGFyZS5jb20vYWpheC9saWJzL2xlYWZsZXQubWFya2VyY2x1c3Rlci8xLjEuMC9sZWFmbGV0Lm1hcmtlcmNsdXN0ZXIuanMiPjwvc2NyaXB0PgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL2NkbmpzLmNsb3VkZmxhcmUuY29tL2FqYXgvbGlicy9sZWFmbGV0Lm1hcmtlcmNsdXN0ZXIvMS4xLjAvTWFya2VyQ2x1c3Rlci5jc3MiLz4KICAgIDxsaW5rIHJlbD0ic3R5bGVzaGVldCIgaHJlZj0iaHR0cHM6Ly9jZG5qcy5jbG91ZGZsYXJlLmNvbS9hamF4L2xpYnMvbGVhZmxldC5tYXJrZXJjbHVzdGVyLzEuMS4wL01hcmtlckNsdXN0ZXIuRGVmYXVsdC5jc3MiLz4KPC9oZWFkPgo8Ym9keT4gICAgCiAgICAKICAgICAgICAgICAgPGRpdiBjbGFzcz0iZm9saXVtLW1hcCIgaWQ9Im1hcF8xMDNjODhlMjRhZGU0MTA5ODdlNThiNmY4NTg0ZDU1ZiIgPjwvZGl2PgogICAgICAgIAo8L2JvZHk+CjxzY3JpcHQ+ICAgIAogICAgCgogICAgICAgICAgICAKICAgICAgICAgICAgICAgIHZhciBib3VuZHMgPSBudWxsOwogICAgICAgICAgICAKCiAgICAgICAgICAgIHZhciBtYXBfMTAzYzg4ZTI0YWRlNDEwOTg3ZTU4YjZmODU4NGQ1NWYgPSBMLm1hcCgKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICdtYXBfMTAzYzg4ZTI0YWRlNDEwOTg3ZTU4YjZmODU4NGQ1NWYnLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAge2NlbnRlcjogWzQwLjY3OTA0NCwtNzMuOTcxNjU1XSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIHpvb206IDEzLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgbWF4Qm91bmRzOiBib3VuZHMsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBsYXllcnM6IFtdLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgd29ybGRDb3B5SnVtcDogZmFsc2UsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBjcnM6IEwuQ1JTLkVQU0czODU3CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0pOwogICAgICAgICAgICAKICAgICAgICAKICAgIAogICAgICAgICAgICB2YXIgdGlsZV9sYXllcl9jN2Q0ZjQwODkxODU0ZjkzODYxNmQ2YTQwMGQ3OTg4NCA9IEwudGlsZUxheWVyKAogICAgICAgICAgICA